In [1]:
# from Feat_eng import Y_train, X_train, Y_test, X_test, c1_train, c2_train, c3_train, c1_test, c2_test, c3_test, Y_c1train, Y_c2train, Y_c3train, Y_c1test, Y_c2test ,Y_c3test

In [1]:
import pickle
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score, RandomizedSearchCV
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import statsmodels.api as sm
import pylab as py
import scipy.stats as stats 
from scipy.stats import norm, skew, probplot
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, RidgeCV, LassoCV, ElasticNetCV
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

In [37]:
from Pred_ZRI_feats import zip_test, zip_train, future_feats, future_targets, Y_train, X_train, Y_test, X_test, c1_train, c2_train, c3_train, c1_test, c2_test, c3_test, Y_c1train, Y_c2train, Y_c3train, Y_c1test, Y_c2test ,Y_c3test, target

In [3]:
target

,RegionName,year,value,geo_id,nonfamily_households,family_households,median_year_structure_built,rent_burden_not_computed,rent_over_50_percent,rent_40_to_50_percent,...,log_value,future_value,male_ratio,white_ratio,black_ratio,asian_ratio,hispanic_ratio,employed_ratio,civil_labor_ratio,Cluster
0,01104,2014,946.100000,01104,3346.0,5526.0,1949.0,158.0,1597.0,523.0,...,6.852348,972.583333,0.477361,0.329594,0.117433,0.010904,0.531703,0.395109,0.469239,1
1,01104,2015,972.583333,01104,3377.0,5407.0,1948.0,146.0,1791.0,466.0,...,6.879956,1058.666667,0.478263,0.319353,0.101512,0.019630,0.549670,0.398254,0.461912,1
2,01104,2016,1058.666667,01104,3432.0,5434.0,1949.0,172.0,1898.0,516.0,...,6.964766,1129.166667,0.467110,0.315878,0.079972,0.017987,0.571669,0.392658,0.449424,1
3,01104,2017,1129.166667,01104,3287.0,5702.0,1946.0,107.0,2032.0,577.0,...,7.029235,1125.083333,0.460292,0.292194,0.078781,0.012735,0.596573,0.388238,0.438968,1
4,01104,2018,1125.083333,01104,3478.0,5599.0,1946.0,75.0,1963.0,502.0,...,7.025612,1145.583333,0.463101,0.275423,0.086447,0.013905,0.604750,0.391816,0.436003,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6042,98499,2014,979.000000,98499,5610.0,6870.0,1973.0,265.0,2355.0,952.0,...,6.886532,995.833333,0.492245,0.467639,0.103251,0.086748,0.211496,0.397328,0.469392,1
6043,98499,2015,995.833333,98499,5736.0,6891.0,1973.0,346.0,2355.0,816.0,...,6.903580,1036.166667,0.505450,0.463689,0.108836,0.087294,0.221283,0.396388,0.464463,1
6044,98499,2016,1036.166667,98499,5670.0,6965.0,1973.0,310.0,2352.0,995.0,...,6.943283,1154.833333,0.502808,0.439341,0.120046,0.093177,0.210382,0.403395,0.459354,1
6045,98499,2017,1154.833333,98499,5149.0,7244.0,1973.0,327.0,2367.0,1006.0,...,7.051711,1257.416667,0.510771,0.425098,0.124402,0.108897,0.206512,0.412859,0.463662,1


In [4]:
future_feats.shape

(2574, 137)

In [5]:
future_targets.shape

(2574, 3)

In [6]:
future_feats

,year,nonfamily_households,family_households,median_year_structure_built,rent_burden_not_computed,rent_over_50_percent,rent_40_to_50_percent,rent_35_to_40_percent,rent_30_to_35_percent,rent_25_to_30_percent,...,est,log_value,male_ratio,white_ratio,black_ratio,asian_ratio,hispanic_ratio,employed_ratio,civil_labor_ratio,Cluster
0,2019,3514.005750,5619.596240,1945.251283,64.221758,2069.917185,500.311260,181.584777,552.064005,465.324405,...,504.060577,7.069693,0.459636,0.263420,0.080712,0.015700,0.624568,0.391020,0.428081,1
1,2020,3550.011500,5640.192481,1944.502565,53.443516,2176.834370,498.622521,161.169554,608.128010,440.648811,...,500.121154,7.113774,0.456172,0.251416,0.074977,0.017495,0.644386,0.390223,0.420160,1
2,2021,3586.017251,5660.788721,1943.753848,42.665274,2283.751555,496.933781,140.754331,664.192015,415.973216,...,496.181731,7.157854,0.452707,0.239413,0.069242,0.019291,0.664204,0.389427,0.412239,1
3,2019,3370.873725,6619.614626,0.000000,311.662957,2014.843245,729.487100,354.197443,677.049498,760.454862,...,248.656208,7.084782,0.478647,0.295612,0.161785,0.047261,0.467141,0.421469,0.459697,1
4,2020,3409.747451,6693.229251,0.000000,344.325913,2032.686491,782.974201,379.394886,696.098996,808.909725,...,246.312417,7.092137,0.479053,0.284688,0.162744,0.046371,0.481499,0.424769,0.457455,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2524,2020,3839.149961,7492.102714,1972.000254,207.124040,647.776627,379.302655,415.846214,846.355676,769.799947,...,330.828118,7.349357,0.490387,0.574766,0.123798,0.083781,0.115791,0.453177,0.460586,1
2525,2021,3756.724942,7609.154072,1972.000381,210.686060,575.664940,377.453983,433.269321,922.533514,858.199920,...,341.242176,7.422146,0.491915,0.569195,0.130759,0.084051,0.116907,0.463085,0.463251,1
2526,2019,5366.584888,7717.252494,1971.750127,533.233881,2199.355425,973.789775,615.147082,906.323242,874.294928,...,1268.491165,7.200888,0.517350,0.426119,0.136041,0.093197,0.214187,0.423036,0.452032,1
2527,2020,5325.169777,7895.504988,1971.500253,608.467762,2169.710849,984.579549,635.294164,988.646483,905.589855,...,1274.982330,7.264961,0.522501,0.418256,0.143377,0.095051,0.214856,0.428323,0.448631,1


In [5]:
with open("Pickle_lasso_all.pkl" , 'rb') as file:  
     lasso_all = pickle.load(file)

In [6]:
with open("Pickle_lasso_all_PCA.pkl" , 'rb') as file:  
     lasso_all_PCA = pickle.load(file)

In [7]:
with open("Pickle_lasso_c1.pkl" , 'rb') as file:  
     lasso_c1 = pickle.load(file)

In [8]:
with open("Pickle_lasso_c1_PCA.pkl" , 'rb') as file:  
     lasso_c1_PCA = pickle.load(file)

In [9]:
with open("Pickle_lasso_c2.pkl" , 'rb') as file:  
     lasso_c2 = pickle.load(file)

In [10]:
with open("Pickle_lasso_c2_PCA.pkl" , 'rb') as file:  
     lasso_c2_PCA = pickle.load(file)

In [11]:
with open("Pickle_lasso_c3.pkl" , 'rb') as file:  
     lasso_c2 = pickle.load(file)

In [12]:
with open("Pickle_lasso_c3_PCA.pkl" , 'rb') as file:  
     lasso_c2_PCA = pickle.load(file)

In [6]:
with open("rf_all.pkl" , 'rb') as file:  
     rf_all = pickle.load(file)

In [14]:
list(future_feats.isnull().sum())

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [20]:
list(zip(future_feats.columns, list(future_feats.max())))

[('year', 2021.0),
 ('nonfamily_households', 24961.959410419437),
 ('family_households', 39191.43550797895),
 ('median_year_structure_built', 2017.7311023079146),
 ('rent_burden_not_computed', 2848.4269941944117),
 ('rent_over_50_percent', 12073.40810342431),
 ('rent_40_to_50_percent', 3186.730045744383),
 ('rent_35_to_40_percent', 2460.3496850851934),
 ('rent_30_to_35_percent', inf),
 ('rent_25_to_30_percent', 3899.5252270303718),
 ('rent_20_to_25_percent', 4483.325450814269),
 ('rent_15_to_20_percent', 4376.319911928115),
 ('rent_10_to_15_percent', inf),
 ('rent_under_10_percent', inf),
 ('total_pop', 148814.68682023592),
 ('median_age', 62.158737450659274),
 ('commuters_by_public_transportation', inf),
 ('households', 46486.47985255736),
 ('median_income', 179693.74810481275),
 ('income_per_capita', 157709.9979141879),
 ('housing_units', 50209.35217695162),
 ('vacant_housing_units', 12429.59346359319),
 ('vacant_housing_units_for_rent', inf),
 ('median_rent', 3743.7767035882207),
 (

In [7]:
print(rf_all.best_params_)
print(rf_all.best_score_)
Y_pred = rf_all.predict(future_feats)
pred_unlog = np.exp(Y_pred)
Y_unlog = future_targets[['ZRI']]
train_RMSE = np.sqrt(mean_squared_error(Y_unlog, pred_unlog))
print('The RMSE is:', train_RMSE)

{'n_estimators': 19, 'min_samples_split': 2, 'min_samples_leaf': 7, 'max_samples': 0.7}
0.9787581366670949
The RMSE is: 191.76328680528468


In [22]:
Y_pred_test = np.exp(rf_all.predict(X_test))

In [23]:
Y_actual = np.exp(Y_test)

In [38]:
zip_test

,RegionName,City
0,01104,Springfield
1,01108,Springfield
2,01109,Springfield
3,01602,Worcester
4,01603,Worcester
...,...,...
920,98125,Seattle
921,98201,Everett
922,98203,Everett
923,98498,Lakewood


In [39]:
diff_df = pd.DataFrame()

In [40]:
diff_df['Zip'] = zip_test['RegionName']
diff_df['City'] = zip_test['City']
diff_df['Pred'] = Y_pred_test
diff_df['Act'] = np.exp(Y_test)

In [46]:
diff_df['Model_Error'] = diff_df['Pred'] - diff_df['Act']

In [47]:
diff_df

,Zip,City,Pred,Act,Diff,Model_Error
0,01104,Springfield,1150.714148,1145.583333,5.130815,5.130815
1,01108,Springfield,1192.499645,1201.500000,-9.000355,-9.000355
2,01109,Springfield,1171.390547,1196.727273,-25.336726,-25.336726
3,01602,Worcester,1462.432603,1401.333333,61.099270,61.099270
4,01603,Worcester,1324.319494,1325.583333,-1.263839,-1.263839
...,...,...,...,...,...,...
920,98125,Seattle,1761.159632,1645.200000,115.959632,115.959632
921,98201,Everett,1454.265146,1450.916667,3.348480,3.348480
922,98203,Everett,1519.896025,1510.916667,8.979359,8.979359
923,98498,Lakewood,1373.151584,1409.000000,-35.848416,-35.848416


In [61]:
comparison = pd.DataFrame()

In [62]:
comparison['Zip'] = future_targets['RegionName']

In [63]:
comparison['Year'] = future_targets['year']

In [64]:
comparison['NoCOVID_ZRI'] = pred_unlog

In [65]:
comparison['COVID_ZRI'] = future_targets[['ZRI']]

In [67]:
comparison['Difference'] = comparison['NoCOVID_ZRI'] - comparison['COVID_ZRI']

In [68]:
comparison = comparison[comparison['Year']==2021]

In [69]:
test = comparison.merge(diff_df, how = 'left', on = 'Zip')

In [75]:
test['Pct'] = (test['Difference'] - test['Model_Error'])/test['Model_Error']

In [79]:
test.sort_values(by = 'Pct', key = abs).head(20)

,Zip,Year,NoCOVID_ZRI,COVID_ZRI,Difference,City,Model_Error,Pct
493,60616,2021,1647.068411,1764.163014,-117.094603,Chicago,-116.089628,0.008657
288,21217,2021,1121.413352,1065.725736,55.687616,Baltimore,52.839951,0.053892
241,15203,2021,1324.409859,1197.623335,126.786525,Pittsburgh,120.197171,0.054821
375,33060,2021,1492.618255,1478.556217,14.062038,Pompano Beach,15.162450,-0.072575
57,06605,2021,1495.417189,1448.258777,47.158412,Bridgeport,42.868472,0.100072
435,44105,2021,726.431181,689.794005,36.637176,Cleveland,41.309214,-0.113099
55,06519,2021,1340.728772,1290.427462,50.301309,New Haven,45.171851,0.113554
594,77082,2021,1268.623845,1208.530694,60.093152,Houston,68.885485,-0.127637
227,14212,2021,817.666935,808.243480,9.423455,Buffalo,10.838821,-0.130583
603,78201,2021,942.003615,913.646211,28.357404,San Antonio,32.776920,-0.134836


In [80]:
test.sort_values(by = 'Pct', key = abs, ascending = False).head(20)

,Zip,Year,NoCOVID_ZRI,COVID_ZRI,Difference,City,Model_Error,Pct
370,32835,2021,1594.376603,1355.068859,239.307744,Orlando,-0.002696,-88764.146207
113,10463,2021,1928.631623,1862.083742,66.547881,New York,0.008664,7679.891538
101,10310,2021,2285.272347,2027.373680,257.898668,New York,0.165075,1561.313911
371,32837,2021,2041.712992,1640.242323,401.470668,Orlando,0.278287,1441.650849
336,28277,2021,1589.566029,1383.890162,205.675867,Charlotte,-0.171162,-1202.647376
640,78754,2021,1502.569728,1300.629486,201.940242,Austin,0.238683,845.059648
198,11420,2021,2486.503137,2245.216902,241.286235,New York,-0.366812,-658.793328
829,97209,2021,2116.814217,1791.355423,325.458794,Portland,0.778037,417.307384
530,63112,2021,818.617806,788.299264,30.318543,Saint Louis,0.091007,332.143601
818,95112,2021,3483.599499,2666.671951,816.927548,San Jose,-2.589536,-316.472606


In [14]:
with open("rf_all_PCA.pkl" , 'rb') as file:  
     rf_all_PCA = pickle.load(file)

In [15]:
with open("rf_cluster1.pkl" , 'rb') as file:  
     rf_c1 = pickle.load(file)

In [16]:
with open("rf_cluster2.pkl" , 'rb') as file:  
     rf_c1 = pickle.load(file)

In [17]:
with open("rf_cluster3.pkl" , 'rb') as file:  
     rf_c1 = pickle.load(file)